In [ ]:
# | default_exp classification.cnnclassifer

In [ ]:
%load_ext jupyter_ai_magics

In [ ]:
# | export
import os

import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
from PIL import Image
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision import models, transforms

In [ ]:
# | export


class TimeSeriesImageDataset(Dataset):
    """Loads time series image data from .png files and corresponding labels from labels.json."""

    def __init__(self, data_dir, resize_shape=(350, 350), transform=None):
        self.data_dir = data_dir
        self.image_files = [f for f in os.listdir(data_dir) if f.endswith(".png")]

        self.labels = self.load_labels()  # Load labels from labels.json
        self.resize_shape = resize_shape
        self.transform = transform if transform else self.default_transform()

    def load_labels(self):
        """Loads labels from a single JSON file."""
        import json

        labels_path = os.path.join(self.data_dir, "labels.json")
        with open(labels_path, "r") as f:
            return json.load(f)

    def default_transform(self):
        """Returns a default transformation pipeline including resizing."""
        return transforms.Compose(
            [
                transforms.Resize(self.resize_shape),
                transforms.ToTensor(),  # Converts to [0, 1] float tensor
            ]
        )

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.image_files[idx])
        img = Image.open(img_path).convert("RGB")  # Convert to 3-channel RGB

        # Apply transformation
        if self.transform:
            img = self.transform(img)

        label = torch.tensor(self.labels[str(idx)], dtype=torch.long)  # Load label from JSON
        return img, label

In [ ]:
# | export


class TimeSeriesDataset(Dataset):
    def __init__(self, data_dir="processed_data", transform=None):
        """
        PyTorch dataset to load time series transformed into image tensors.

        Args:
            data_dir (str): Directory containing the saved .pt files.
            transform (callable, optional): Optional transform to apply to images.
        """
        self.data_dir = data_dir
        self.transform = transform
        self.files = sorted([f for f in os.listdir(data_dir) if f.endswith(".pt")])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.files[idx])
        sample = torch.load(file_path)

        image, label = sample["image"], sample["label"]

        if self.transform:
            image = self.transform(image)  # Apply any transformations (e.g., normalization)

        return image, label

In [ ]:
# | export


class TimeSeriesDataModule(pl.LightningDataModule):
    def __init__(
        self,
        data_dir="processed_data",
        batch_size=64,
        num_workers=4,
        val_split=0.1,
        test_split=0.1,
        resize_shape=(350, 350),
    ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.val_split = val_split
        self.test_split = test_split
        self.resize_shape = resize_shape

    def setup(self, stage=None):
        """Randomly split dataset into train, validation, and test sets."""
        full_dataset = TimeSeriesDataset(self.data_dir)
        total_size = len(full_dataset)

        val_size = int(self.val_split * total_size)
        test_size = int(self.test_split * total_size)
        train_size = total_size - val_size - test_size

        # ✅ Randomly split dataset
        self.train_dataset, self.val_dataset, self.test_dataset = random_split(
            full_dataset,
            [train_size, val_size, test_size],
            generator=torch.Generator().manual_seed(42),  # Ensure reproducibility
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
        )

In [ ]:
# | export


class TSImageClassifier(pl.LightningModule):
    def __init__(
        self,
        model_name="convnext_tiny",
        num_classes=10,
        hidden_feature=512,
        lr=1e-3,
        freeze_backbone=True,
    ):
        super().__init__()
        self.save_hyperparameters()

        # Load model
        self.pretrained_model = self._load_model(model_name)

        # Freeze backbone if required
        if freeze_backbone:
            for param in self.pretrained_model.parameters():
                param.requires_grad = False

        # Modify classifier for custom classes
        self._modify_classifier(hidden_feature, num_classes)

        # Loss function
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr

        # Metrics (computed per batch)
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(
            task="multiclass", num_classes=num_classes, average="macro"
        )
        self.auc = torchmetrics.AUROC(task="multiclass", num_classes=num_classes)

    def _load_model(self, model_name):
        """Load a pretrained model dynamically."""
        model_dict = {
            "convnext_tiny": models.convnext_tiny(weights="IMAGENET1K_V1"),
            "efficientnet_b0": models.efficientnet_b0(weights="IMAGENET1K_V1"),
            "swin_v2_s": models.swin_v2_s(weights="IMAGENET1K_V1"),
            "resnet50": models.resnet50(weights="IMAGENET1K_V1"),
        }
        if model_name not in model_dict:
            raise ValueError(
                f"Model '{model_name}' is not supported. Choose from {list(model_dict.keys())}."
            )
        return model_dict[model_name]

    def _modify_classifier(self, hidden_feature, num_classes):
        """Modify classifier head for different models."""
        if hasattr(self.pretrained_model, "classifier"):
            in_features = self.pretrained_model.classifier[-1].in_features
            self.pretrained_model.classifier[-1] = nn.Sequential(
                nn.Linear(in_features, hidden_feature),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                # nn.Linear(hidden_feature, hidden_feature),
                # nn.ReLU(),
                # nn.Dropout(p=0.3),
                nn.Linear(hidden_feature, num_classes),
            )
        elif hasattr(self.pretrained_model, "fc"):
            in_features = self.pretrained_model.fc.in_features
            self.pretrained_model.fc = nn.Sequential(
                nn.Linear(in_features, hidden_feature),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                # nn.Linear(hidden_feature, hidden_feature),
                # nn.ReLU(),
                # nn.Dropout(p=0.3),
                nn.Linear(hidden_feature, num_classes),
            )

    def forward(self, x):
        return self.pretrained_model(x)

    def compute_metrics(self, logits, y, prefix):
        """Compute Accuracy, F1 Score, and AUC for a given batch."""
        preds = torch.argmax(logits, dim=1)
        acc = self.accuracy(preds, y)
        f1 = self.f1_score(preds, y)
        auc = self.auc(logits, y)

        self.log_dict(
            {
                f"{prefix}_accuracy": acc,
                f"{prefix}_f1": f1,
                f"{prefix}_auc": auc,
            },
            prog_bar=True,
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)

        # Log metrics
        self.log("train_loss", loss, prog_bar=True)
        self.compute_metrics(logits, y, "train")

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)

        # Log metrics
        self.log("val_loss", loss, prog_bar=True)
        self.compute_metrics(logits, y, "val")

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)

        # Log metrics
        self.log("test_loss", loss, prog_bar=True)
        self.compute_metrics(logits, y, "test")

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        return [optimizer], [scheduler]

In [ ]:
# | export
# Compute dynamic convolution parameters


def compute_conv_params(input_size, output_size):
    stride = input_size // output_size
    kernel_size = (stride * 2) if stride > 1 else 3
    padding = (kernel_size - stride) // 2
    return kernel_size, stride, padding


# Preprocessing: Channel Reduction & Spatial Downsampling


class ChannelReducerAndDownscaler(nn.Module):
    def __init__(self, in_channels=164, reduced_channels=3, input_size=500, output_size=250):
        super(ChannelReducerAndDownscaler, self).__init__()
        kernel_size, stride, padding = compute_conv_params(input_size, output_size)

        # Reduce Channels
        self.channel_reducer = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(32, reduced_channels, kernel_size=1),
            nn.ReLU(),
        )

        # Downscale Spatial Dimensions using Conv instead of Linear
        self.spatial_downscaler = nn.Conv2d(
            reduced_channels, reduced_channels, kernel_size, stride, padding
        )

    def forward(self, x):
        x = self.channel_reducer(x)  # Reduce channels
        x = self.spatial_downscaler(x)  # Downscale
        return x  # Output: (batch, channels, output_size, output_size)


# Combined Model: Preprocessing + Classification


class TSNDTensorClassifier(pl.LightningModule):
    def __init__(
        self,
        model_name="convnext_tiny",
        num_classes=10,
        hidden_feature=256,
        lr=5e-4,
        freeze_backbone=True,
        in_channels=164,
        reduced_channels=3,
        input_size=500,
        output_size=250,
    ):
        super().__init__()
        self.save_hyperparameters()

        # Add Preprocessing (Channel Reduction & Downsampling)
        self.preprocessor = ChannelReducerAndDownscaler(
            in_channels, reduced_channels, input_size, output_size
        )

        # Load Pretrained Model
        self.pretrained_model = self._load_model(model_name)

        # Freeze Backbone If Required
        if freeze_backbone:
            for param in self.pretrained_model.parameters():
                param.requires_grad = False

        # Modify Classifier
        self._modify_classifier(hidden_feature, num_classes)

        # Loss and Metrics
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr
        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.f1_score = torchmetrics.F1Score(
            task="multiclass", num_classes=num_classes, average="macro"
        )
        self.auc = torchmetrics.AUROC(task="multiclass", num_classes=num_classes)

    def _load_model(self, model_name):
        """Load a pretrained model dynamically."""
        model_dict = {
            "convnext_tiny": models.convnext_tiny(weights="IMAGENET1K_V1"),
            "efficientnet_b0": models.efficientnet_b0(weights="IMAGENET1K_V1"),
            "swin_v2_s": models.swin_v2_s(weights="IMAGENET1K_V1"),
            "resnet50": models.resnet50(weights="IMAGENET1K_V1"),
        }
        if model_name not in model_dict:
            raise ValueError(
                f"Unsupported model '{model_name}'. Choose from {list(model_dict.keys())}."
            )
        return model_dict[model_name]

    def _modify_classifier(self, hidden_feature, num_classes):
        """Modify classifier head for different models."""
        if hasattr(self.pretrained_model, "classifier"):
            in_features = self.pretrained_model.classifier[-1].in_features
            self.pretrained_model.classifier[-1] = nn.Sequential(
                nn.Linear(in_features, hidden_feature),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                nn.Linear(hidden_feature, num_classes),
            )
        elif hasattr(self.pretrained_model, "fc"):
            in_features = self.pretrained_model.fc.in_features
            self.pretrained_model.fc = nn.Sequential(
                nn.Linear(in_features, hidden_feature),
                nn.ReLU(),
                nn.Dropout(p=0.3),
                nn.Linear(hidden_feature, num_classes),
            )

    def forward(self, x):

        x = self.preprocessor(x)  # Apply preprocessing first
        x = self.pretrained_model(x)  # Pass through classifier
        return x

    def compute_metrics(self, logits, y, prefix):
        preds = torch.argmax(logits, dim=1)
        acc, f1, auc = self.accuracy(preds, y), self.f1_score(preds, y), self.auc(logits, y)
        self.log_dict(
            {f"{prefix}_accuracy": acc, f"{prefix}_f1": f1, f"{prefix}_auc": auc}, prog_bar=True
        )

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("train_loss", loss, prog_bar=True)
        self.compute_metrics(logits, y, "train")
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("val_loss", loss, prog_bar=True)
        self.compute_metrics(logits, y, "val")

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("test_loss", loss, prog_bar=True)
        self.compute_metrics(logits, y, "test")

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        return [optimizer], [scheduler]

In [ ]:
# | hide


# Example Usage
if __name__ == "__main__":
    x = torch.rand(8, 145, 700, 700)  # Example Batch: (Batch, Channels, Height, Width)

    model = TSNDTensorClassifier(
        model_name="efficientnet_b0",
        num_classes=5,
        in_channels=x.shape[1],
        reduced_channels=3,
        input_size=x.shape[-1],
        output_size=250,
    )

    output = model(x)
    print(output.shape)  # Expected: (Batch, num_classes)

torch.Size([8, 5])


In [ ]:
## example

In [ ]:
# Load & preprocess data
df = pd.read_parquet("data/m4_preprocessed.parquet")
df.sort_values("no_of_datapoints", inplace=True)

# Try on smaller dataset
df = df[df.no_of_datapoints <= 300]

df.drop(columns=["no_of_datapoints"], inplace=True)
# ts_series = df.drop(["best_model"], axis=1).iloc[20].dropna()
df = df.reset_index(drop=True)

In [ ]:
df

,1,2,3,4,5,6,7,8,9,10,...,9925,9926,9927,9928,9929,9930,9931,9932,9933,best_model
0,1815.0,1618.0,1598.0,1570.0,1520.0,1600.0,1570.0,1494.0,1367.0,1520.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoRegressive
1,1189.0,1225.0,1233.0,1264.0,1445.0,1595.0,1718.0,1668.0,1700.0,1712.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoTheta
2,1120.0,1060.0,1510.0,1630.0,1780.0,1520.0,1530.0,1700.0,1550.0,1910.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoMFLES
3,3260.0,2770.0,2890.0,3310.0,3440.0,3700.0,3900.0,3870.0,3490.0,3010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CES
4,2590.0,2510.0,2980.0,2370.0,2220.0,2340.0,2010.0,1960.0,1760.0,2000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoMFLES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30218,1498.0,1503.0,1516.0,1534.0,1536.0,1530.0,1556.0,1565.0,1558.0,1562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CES
30219,3049.0,3056.0,3083.0,3084.0,3100.0,3157.0,3153.0,3162.0,3135.0,3093.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoETS
30220,2470.0,2390.0,2400.0,2370.0,2390.0,2420.0,2420.0,2390.0,2450.0,2390.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoARIMA
30221,6430.0,6340.0,6390.0,6300.0,6360.0,6460.0,6520.0,6480.0,6400.0,6430.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CES


In [ ]:
from ts.tsfeatures.ts2image import transform_tensor2img

In [ ]:
transform_tensor2img(
    df, data_dir="model_classification", categorical_label=True, label_col="best_model"
)

Transforming & Saving (X, y): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 30223/30223 [03:14<00:00, 155.35it/s]


In [ ]:
import json

with open("model_classification/classes.json", "r") as file:
    classes = json.load(file)
ds = TimeSeriesDataModule(data_dir="model_classification", batch_size=32, num_workers=24)
model = TSImageClassifier(
    model_name="efficientnet_b0",
    num_classes=len(classes),
    hidden_feature=512,
    lr=3e-3,
    freeze_backbone=False,
)

In [ ]:
import wandb
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(
    project="ts-classification", name="cnn.model=efficientnet_b0.ds=model_classifier"
)
wandb_logger.experiment.config["model"] = "efficientnet_b0"
wandb_logger.experiment.config["ds"] = "model_classifier"
wandb_logger.experiment.config["finetune"] = False

In [ ]:
trainer = pl.Trainer(
    # logger=wandb_logger,
    accelerator="auto",
    devices=[0],
    min_epochs=1,
    max_epochs=100,
    enable_checkpointing=True,
    callbacks=[
        pl.callbacks.EarlyStopping("val_loss", patience=5, verbose=False),
    ],
)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [ ]:
ckpt_path = "model_checkpoints/cnn_model_classification.ckpt"
finetune = False
if finetune:
    trainer.fit(model, ds, ckpt_path=ckpt_path)
else:
    trainer.fit(model, ds)

In [ ]:
trainer.save_checkpoint("model_checkpoints/cnn_model_classification.ckpt")

In [ ]:
trainer.test(model, ds);

Testing: |                                                                                                    …

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.22667108476161957    │
│         test_auc          │    0.48773670196533203    │
│          test_f1          │    0.05385681614279747    │
│         test_loss         │     1.896951675415039     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
df

,1,2,3,4,5,6,7,8,9,10,...,9925,9926,9927,9928,9929,9930,9931,9932,9933,best_model
0,1815.0,1618.0,1598.0,1570.0,1520.0,1600.0,1570.0,1494.0,1367.0,1520.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoRegressive
1,1189.0,1225.0,1233.0,1264.0,1445.0,1595.0,1718.0,1668.0,1700.0,1712.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoTheta
2,1120.0,1060.0,1510.0,1630.0,1780.0,1520.0,1530.0,1700.0,1550.0,1910.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoMFLES
3,3260.0,2770.0,2890.0,3310.0,3440.0,3700.0,3900.0,3870.0,3490.0,3010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CES
4,2590.0,2510.0,2980.0,2370.0,2220.0,2340.0,2010.0,1960.0,1760.0,2000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoMFLES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30218,1498.0,1503.0,1516.0,1534.0,1536.0,1530.0,1556.0,1565.0,1558.0,1562.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CES
30219,3049.0,3056.0,3083.0,3084.0,3100.0,3157.0,3153.0,3162.0,3135.0,3093.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoETS
30220,2470.0,2390.0,2400.0,2370.0,2390.0,2420.0,2420.0,2390.0,2450.0,2390.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AutoARIMA
30221,6430.0,6340.0,6390.0,6300.0,6360.0,6460.0,6520.0,6480.0,6400.0,6430.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CES
